### 1.什么是Command
简单来说，Command就像是给节点赋予了”决策权“，它不仅能够处理数据、更新状态，还能直接决定下一步要执行哪个节点，而不需要依赖外部的边或路由函数。
传统的LangGraph工作流中，节点和边的职责是分离的：节点负责处理逻辑和更新状态，边负责控制流程。但在某些场景下，我们需要在节点内部同时完成状态更新和流程控制，这时候Command就派上用场了。
Command本质上是一个特殊的对象，节点函数可以返回它来同时执行两个操作：
1. 更新Graph的状态（通过update参数）
2. 指定下一个执行的节点（通过goto参数）
### 2.为什么需要Command
#### 2.1 传统方式的局限
传统的方式节点和路由逻辑是分离的。我们需要先更新状态，然后在另一个函数中读取状态来决定路由。当我们的逻辑变得非常复杂时，代码会变得分散，难以维护。
#### 2.2 Command的优势
使用Command后，同样的功能只需要一个节点函数就能完成：
```python
from langgraph.types import Command

def my_node(state: State) -> Command[Literal["success_node", "failure_node"]]:
    # 在一个函数中同时完成状态更新和路由决策
    if some_condition:
        return Command(
            update={"status": "success"},  # 更新状态
            goto="success_node"              # 决定路由
        )
    else:
        return Command(
            update={"status": "failure"},    # 更新状态
            goto="failure_node"              # 决定路由
        )

# 只需要添加节点，不需要额外的路由函数
graph.add_node("my_node", my_node)
```
使用Command后，在一个节点函数内部就能同时完成状态更新和路由决策，这样逻辑集中在一个地方，代码更清晰、简洁、易于维护。
### 3.Command参数详解
Command类的使用关键，主要是四个参数的运用。
#### 3.1 update参数
update参数用于更新Graph的状态。它可以是一个字典，包含要更新的字段和对应的值。
```python
def my_node(state: State) -> Command:
    return Command(
        update={
            "user_info": {"name": "张三", "age": 25},
            "status": "processed"
        },
        goto="next_node"
    )
```
这里，update参数会更新State中的user_info和status字段。
#### 3.2 goto参数
goto参数用于指定下一个要执行的节点。这是Command最核心的功能之一。它支持多种形式，可以传入节点名称（字符串）、节点序列、Send对象等。
##### 3.2.1 节点名称（字符串）
最简单的形式是传入一个节点名称（字符串）。
```python
from langgraph.types import Command
from typing import Literal

def my_node(state: State) -> Command[Literal["next_node"]]:
    return Command(
        update={"status": "done"},
        goto="next_node"  # 跳转到next_node节点
    )
```
##### 3.2.2 节点序列
如果需要顺序执行多个节点，可以传入一个节点序列：
```python
def my_node(state: State) -> Command[Literal["next_node", "next_node2"]]:
    return Command(
        update={"status": "done"},
        goto=["next_node", "next_node2"]  # 先执行next_node，再执行next_node2
    )
```
##### 3.2.3 Send对象
goto参数还可以接收Send对象，这样就能在Command中实现复杂的动态路由。
```python
from langgraph.types import Send

def distribute_tasks(state: State) -> Command:
    """将任务分发到多个处理节点"""
    tasks = state["tasks"]
    return Command(
        update={"status": "distributed"},
        goto=[Send("process_task", {"task": task}) for task in tasks]
    )
```
当goto是节点序列时，这些节点会按照顺序执行；如果传入的是Send对象列表，LangGraph会尝试并行执行这些Send指向的节点。
#### 3.3 graph参数
graph参数用来指定要发送命令的目标Graph，默认值是None，表示当前Graph。这个参数主要用于子图场景。
需要说明的是，graph参数和goto参数要配合理解，即goto参数指向的节点名称，必须是graph参数指定的Graph内部的节点。也就是说：
- 如果graph=None（默认值），goto指向的是当前Graph的节点。
- 如果graph=Command.PARENT，goto指向的是父Graph的节点（必须是父图中存在的节点）。
```python
from langgraph.types import Command
from typing import Literal

def subgraph_node(state: State) -> Command[Literal["parent_node"]]:
    # 在子图中执行某些逻辑后，跳转到父图的节点
    return Command(
        update={"result": "completed"},
        goto="parent_node",  # 这个节点必须是父图中存在的节点
        graph=Command.PARENT  # 告诉LangGraph要跳转到父图，而不是当前子图
    )
```
#### 3.4 resume参数
resume参数通常与interrupt()函数配合使用，主要用于Human in the loop，中断Graph执行、等待外部输入或异步操作完成的场景。
这个后续再详细了解。
### 4.Command的典型使用场景
#### 4.1.动态控制流
#### 4.2.工具中的状态更新
#### 4.3.子图中的导航
#### 4.4.多智能体交接
### 5.使用Command的注意事项
#### 5.1.类型提示的重要性
#### 5.2.update参数的合并规则
#### 5.3.子图中使用Command.PARENT的编译注意事项
- 编译时验证：由于目标节点在父图中，子图编译时无法验证该节点是否存在，LangGraph会要求子图必须有明确的出口（边
- 临时END边：我们需要添加一个临时的END边来通过编译验证，但实际运行时会被Command覆盖，不会真正执行到END
```python
# 子图中的节点使用Command跳转到父图
defsubgraph_node(state: State) -> Command:
    return Command(
        update={"status": "done"},
        goto="parent_node",
        graph=Command.PARENT
    )

# 构建子图
subgraph = StateGraph(State)
subgraph.add_node("subgraph_node", subgraph_node)
subgraph.add_edge(START, "subgraph_node")
# 注意：需要添加临时的END边来通过编译验证
# 实际运行时会被Command覆盖，不会真正执行到END
subgraph.add_edge("subgraph_node", END)
subgraph_app = subgraph.compile()
```


